Hacer las grafocas para las 6 tandas de datos que mando Julian. 

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import os
from glob import glob

# --- Parámetros de detección de picos ---
UMBRAL_ALTURA_RELATIVA = 0.1
PROMINENCE_MINIMA = 0.05
DISTANCIA_MINIMA_PUNTOS = 5
NUM_MAX_HARMONICOS = 5  # Número máximo de armónicos a mostrar

def cargar_datos(ruta_archivo):
    with open(ruta_archivo, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    data_lines = [line for line in lines if not line.startswith('#')]
    temp_path = 'temp_data.tsv'
    with open(temp_path, 'w', encoding='utf-8') as f:
        f.writelines(data_lines)
    df = pd.read_csv(temp_path, sep='\t', engine='python')
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df.dropna()
    return df

def calcular_frecuencia_muestreo(tiempos):
    if len(tiempos) > 1:
        frecuencia = 1 / np.mean(np.diff(tiempos))
        print(f"Frecuencia de muestreo estimada: {frecuencia:.2f} Hz")
        return frecuencia
    else:
        print("No hay suficientes puntos de tiempo para estimar la frecuencia de muestreo.")
        return None

def aplicar_fft(senal, frecuencia_muestreo):
    N = len(senal)
    transformada = np.fft.fft(senal)
    amplitudes = np.abs(transformada)
    frecuencias = np.fft.fftfreq(N, d=1/frecuencia_muestreo)
    return frecuencias, amplitudes, N

def graficar_resultados(tiempo, angulos, espectros, nombre_archivo, output_dir):
    nombres = ['1', '2', '3']
    colores = ['skyblue', 'magenta', 'purple']
    plt.figure(figsize=(12, 6))

    # Ángulo vs Tiempo
    plt.subplot(2, 1, 1)
    for i, angulo in enumerate(angulos):
        plt.plot(tiempo, angulo, label=f'Péndulo {nombres[i]}', color=colores[i])
    plt.title('Ángulo vs Tiempo')
    plt.xlabel('Tiempo (s)')
    plt.ylabel('Ángulo (rad)')
    plt.legend()
    plt.grid(True)

    # Espectro de frecuencias
    plt.subplot(2, 1, 2)
    for i, (frec, amp, N) in enumerate(espectros):
        frec_pos = frec > 0
        frec_plot = frec[frec_pos]
        amp_plot = amp[frec_pos] * 2 / N

        plt.plot(frec_plot, amp_plot, label=f'Péndulo {nombres[i]}', color=colores[i])

        # Detección de picos (solo el más alto)
        peaks, properties = find_peaks(
            amp_plot,
            height=np.max(amp_plot) * UMBRAL_ALTURA_RELATIVA,
            prominence=PROMINENCE_MINIMA,
            distance=DISTANCIA_MINIMA_PUNTOS
        )

        print(f"\n🎯 Frecuencia más destacada para Péndulo {nombres[i]}:")
        if len(peaks) > 0:
            alturas = properties['peak_heights']
            idx_max = np.argmax(alturas)
            p = peaks[idx_max]

            freq_val = frec_plot[p]
            amp_val = amp_plot[p]
            print(f" - {freq_val:.2f} Hz (amplitud: {amp_val:.4f})")

            # Marca el pico principal
            plt.plot(freq_val, amp_val, 'o', color=colores[i])
            plt.text(freq_val, amp_val, f'{freq_val:.2f} Hz', fontsize=8,
                     ha='left', va='bottom', rotation=45)

            # Dibujar líneas verticales de los armónicos
            f_max = frec_plot.max()
            for n in range(2, NUM_MAX_HARMONICOS + 1):
                armonico = n * freq_val
                if armonico < f_max:
                    plt.axvline(armonico, color=colores[i], linestyle='--', alpha=0.5)
                    plt.text(armonico, plt.ylim()[1]*0.9, f'{n}ω', color=colores[i],
                             rotation=90, va='top', ha='center', fontsize=8)
        else:
            print(" - No se detectaron picos significativos.")

    plt.title('Espectro de Frecuencias')
    plt.xlabel('Frecuencia (Hz)')
    plt.ylabel('Amplitud')
    plt.grid(True)
    plt.legend()

    plt.tight_layout()

    os.makedirs(output_dir, exist_ok=True)
    base_name = os.path.splitext(os.path.basename(nombre_archivo))[0]
    output_file = os.path.join(output_dir, f"{base_name}.pdf")
    plt.savefig(output_file, format='pdf', dpi=300)
    print(f"\n📁 Gráfico guardado como: {output_file}")
    plt.close()

# ---- PROCESAMIENTO DE TODOS LOS ARCHIVOS ----

archivos_tsv = glob("*.tsv")
output_dir = "figures"

for archivo in archivos_tsv:
    print(f"\n📄 Procesando archivo: {archivo}")
    try:
        df = cargar_datos(archivo)

        if df.shape[1] < 4:
            print("⚠️  El archivo debe contener al menos 4 columnas: tiempo + 3 ángulos.")
            continue

        tiempos = df.iloc[:, 0].values
        angulos = [df.iloc[:, i].values for i in range(1, 4)]

        duracion_maxima = 30
        indices_validos = tiempos <= duracion_maxima
        tiempos = tiempos[indices_validos]
        angulos = [a[indices_validos] for a in angulos]

        frecuencia = calcular_frecuencia_muestreo(tiempos)

        if frecuencia:
            espectros = [aplicar_fft(angulo, frecuencia) for angulo in angulos]
            graficar_resultados(tiempos, angulos, espectros, archivo, output_dir)
        else:
            print("FFT no realizada por falta de frecuencia de muestreo.")
    except Exception as e:
        print(f"❌ Error procesando el archivo {archivo}: {e}")



📄 Procesando archivo: 010_26.tsv
Frecuencia de muestreo estimada: 18.41 Hz

🎯 Frecuencia más destacada para Péndulo 1:
 - 0.87 Hz (amplitud: 1.6387)

🎯 Frecuencia más destacada para Péndulo 2:
 - 0.83 Hz (amplitud: 4.2784)

🎯 Frecuencia más destacada para Péndulo 3:
 - 1.23 Hz (amplitud: 0.5369)

📁 Gráfico guardado como: figures/010_26.pdf

📄 Procesando archivo: 111_11.tsv
Frecuencia de muestreo estimada: 18.46 Hz

🎯 Frecuencia más destacada para Péndulo 1:
 - 1.17 Hz (amplitud: 7.1602)

🎯 Frecuencia más destacada para Péndulo 2:
 - 0.83 Hz (amplitud: 5.7789)

🎯 Frecuencia más destacada para Péndulo 3:
 - 1.20 Hz (amplitud: 4.5985)

📁 Gráfico guardado como: figures/111_11.pdf

📄 Procesando archivo: 001_16.tsv
Frecuencia de muestreo estimada: 18.44 Hz

🎯 Frecuencia más destacada para Péndulo 1:
 - 1.33 Hz (amplitud: 6.3446)

🎯 Frecuencia más destacada para Péndulo 2:
 - 0.83 Hz (amplitud: 0.5109)

🎯 Frecuencia más destacada para Péndulo 3:
 - 1.20 Hz (amplitud: 1.4467)

📁 Gráfico guard

In [5]:
import pandas as pd
import os
from glob import glob

def cargar_datos(ruta_archivo):
    with open(ruta_archivo, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    data_lines = [line for line in lines if not line.startswith('#')]
    temp_path = 'temp_data.tsv'
    with open(temp_path, 'w', encoding='utf-8') as f:
        f.writelines(data_lines)
    df = pd.read_csv(temp_path, sep='\t', engine='python')
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df.dropna()

# Crear carpeta de salida
output_dir = "datos_filtrados"
os.makedirs(output_dir, exist_ok=True)

# Obtener lista de archivos
archivos_tsv = sorted(glob("*.tsv"))
archivos_tsv = [f for f in archivos_tsv if f != "temp_data.tsv"]

# Procesar cada archivo
for archivo in archivos_tsv:
    print(f"\n📄 Procesando: {archivo}")
    try:
        df = cargar_datos(archivo)

        if df.shape[1] < 1:
            print("⚠️  El archivo no tiene suficientes columnas.")
            continue

        tiempos = df.iloc[:, 0]
        t_min, t_max = tiempos.min(), tiempos.max()
        print(f" - Tiempo disponible: {t_min:.2f} s a {t_max:.2f} s")

        # Ingreso del t0
        while True:
            try:
                t0 = float(input(f"⏱️  Ingrese el t0 para '{archivo}': "))
                break
            except ValueError:
                print("❌ Ingrese un número válido.")

        # Filtrar desde t0
        df_filtrado = df[tiempos >= t0].reset_index(drop=True)

        # Guardar archivo limpio
        nombre_base = os.path.splitext(os.path.basename(archivo))[0]
        salida = os.path.join(output_dir, f"{nombre_base}_filtrado.tsv")
        df_filtrado.to_csv(salida, sep='\t', index=False)
        print(f"✅ Datos filtrados guardados en: {salida}")
    except Exception as e:
        print(f"❌ Error procesando {archivo}: {e}")



📄 Procesando: 001_16.tsv
 - Tiempo disponible: 0.05 s a 104.92 s


⏱️  Ingrese el t0 para '001_16.tsv':  2.5


✅ Datos filtrados guardados en: datos_filtrados/001_16_filtrado.tsv

📄 Procesando: 001_66.tsv
 - Tiempo disponible: 0.05 s a 63.09 s


⏱️  Ingrese el t0 para '001_66.tsv':  2.5


✅ Datos filtrados guardados en: datos_filtrados/001_66_filtrado.tsv

📄 Procesando: 010_15.tsv
 - Tiempo disponible: 0.05 s a 61.34 s


...

✅ Datos filtrados guardados en: datos_filtrados/010_66_filtrado.tsv

📄 Procesando: 101_16.tsv
 - Tiempo disponible: 0.05 s a 56.24 s


⏱️  Ingrese el t0 para '101_16.tsv':  3


✅ Datos filtrados guardados en: datos_filtrados/101_16_filtrado.tsv

📄 Procesando: 111_11.tsv
 - Tiempo disponible: 0.05 s a 93.12 s


⏱️  Ingrese el t0 para '111_11.tsv':  16


✅ Datos filtrados guardados en: datos_filtrados/111_11_filtrado.tsv
